# Overview

This notebook goes through a fixed-base and chained bilateral indices methods demonstrated by Aizcorbe (2014) in 2.1. This this example Ana Aizcorbe uses US National Income and Product Accounts data from the BEA to demonstrate how the Laspeyres, Paasche, and Fisher behave differently when you use a fixed base versus when they are chained.

In [1]:
import pandas as pd
import numpy as np

In [58]:
#if not run before, use the helper to clean the data and make it analysis ready
from ..src.nipa_helper import clean_nipa_data

clean_nipa_data()

ImportError: attempted relative import with no known parent package

## Explore the data

While the raw data is a tad hard to work with (and looks awkward in Table 2.3), this data is provided in cleaned `csv` format in the `data\silver` folder

In [11]:
df_expenditures = pd.read_csv("../data/bronze/2000-2010 US NIPA data - nominal spending.csv")
df_expenditures

,Years,Motor Vehicles and Parts,Furnishings and Durable Household Equipment,Recreational Goods and Vehicles,Other Durable Goods
0,2000,363.2,208.1,234.1,110.4
1,2001,383.3,214.9,239.8,108.4
2,2002,401.3,225.9,251.5,113.4
3,2003,401.0,231.8,265.7,121.4
4,2004,403.9,247.0,290.5,131.5
5,2005,408.2,261.3,312.8,141.1
6,2006,394.8,271.5,334.1,154.6
7,2007,399.9,271.3,349.4,167.8
8,2008,339.3,257.9,344.0,167.7
9,2009,316.5,235.3,316.6,161.2


In [18]:
df_expenditure_new = df_expenditures.melt(id_vars=['Years'], var_name='product_name', value_name='spending')
df_expenditure_new.head()

,Years,product_name,spending
0,2000,Motor Vehicles and Parts,363.2
1,2001,Motor Vehicles and Parts,383.3
2,2002,Motor Vehicles and Parts,401.3
3,2003,Motor Vehicles and Parts,401.0
4,2004,Motor Vehicles and Parts,403.9


In [3]:
df_price_indices = pd.read_csv("../data/bronze/2000-2010 US NIPA data - price indexes.csv")
df_price_indices

,Years,Motor Vehicles and Parts,Furnishings and Durable Household Equipment,Recreational Goods and Vehicles,Other Durable Goods
0,2000,102.00,108.12,136.29,105.23
1,2001,102.40,106.27,126.99,105.56
2,2002,101.86,104.08,118.78,103.61
3,2003,99.08,101.16,111.28,101.91
4,2004,98.40,99.91,105.76,101.54
5,2005,100.00,100.00,100.00,100.00
6,2006,100.11,99.60,93.79,101.80
7,2007,99.62,98.89,87.16,105.53
8,2008,97.82,98.14,82.65,109.09
9,2009,98.16,97.82,77.35,110.40


In [51]:
df_price_new = df_price_indices.melt(id_vars=['Years'], var_name='product_name', value_name='index')
df_price_new.head()

,Years,product_name,index
0,2000,Motor Vehicles and Parts,102.00
1,2001,Motor Vehicles and Parts,102.40
2,2002,Motor Vehicles and Parts,101.86
3,2003,Motor Vehicles and Parts,99.08
4,2004,Motor Vehicles and Parts,98.40


In [16]:
# df_price_new.join(df_expenditure_new,on=['Years','product_name'])

In [66]:
df_merged = pd.merge(df_expenditure_new, df_price_new, on='product_name', how='inner').drop('Years_y', axis='columns')
df_merged.rename(columns={"Years_x":"Time", "spending":"quantity"}, inplace=True)
df_merged.head()

,Time,product_name,quantity,index
0,2000,Motor Vehicles and Parts,363.2,102.00
1,2000,Motor Vehicles and Parts,363.2,102.40
2,2000,Motor Vehicles and Parts,363.2,101.86
3,2000,Motor Vehicles and Parts,363.2,99.08
4,2000,Motor Vehicles and Parts,363.2,98.40


In [ ]:
def clean_nipa_data():
    """
    Cleans the NIPA data used in Aizcorbe's example 2.1

    Takes two separate csv's in bronze and creates an analysis ready
    dataset
    """
    # import the expenditure data and format it
    df_expenditures = pd.read_csv("../data/bronze/2000-2010 US NIPA data - nominal spending.csv")
    df_expenditure_new = df_expenditures.melt(id_vars=['Years'], var_name='product_name', value_name='spending')

    # import the prices(indices) data and format it
    df_price_indices = pd.read_csv("../data/bronze/2000-2010 US NIPA data - price indexes.csv")
    df_price_new = df_price_indices.melt(id_vars=['Years'], var_name='product_name', value_name='index')
    
    # merge the two together
    pd.merge(df_expenditure_new, df_price_new, on='product_name', how='inner').drop('Years_y', axis='columns')
    df_merged.rename(columns={"Years_x":"Time", "spending":"quantity"}, inplace=True)



## Demonstration of using numpy array to do example in table 2.1

In [19]:
p0 = np.array([6,4])
p1 = np.array([7,10])
q0 = np.array([200,200])
q1 = np.array([600,100])

Laspeyres

In [20]:
np.sum(p1 * q0) / np.sum(p0 * q0)

1.7

Paasche

In [22]:
np.sum(p1 * q1) / np.sum(p0 * q1)

1.3

## Examples using functions from [`PriceIndexCalc`](https://github.com/drrobotk/PriceIndexCalc/tree/main)

In [40]:
def laspeyres(
    p0: np.array, 
    p1: np.array,
    q0: np.array,
) -> float:
    """
    Laspeyres bilateral index, using price and base quantity information.
    
    .. math::
        \\text{Laspeyres} = \\frac{\\sum_{i=1}^{n} p_i}{\\sum_{i=1}^{n} p_0}

    :param p0: Base price vector.
    :param p1: Current price vector.
    :param q0: Base quantity vector.
    """
    return lowe(p0, p1, q0)

def paasche(
    p0: np.array, 
    p1: np.array,
    q1: np.array,
) -> float:
    """Paasche bilateral index, using price and current quantity information."""
    return lowe(p0, p1, q1)

def lowe(
    p0: np.array, 
    p1: np.array,
    q: np.array,
) -> float:
    """
    Lowe bilateral index, using price and arbitrary quantity information.
    
    .. math::
        \\text{Lowe} = \\frac{\\sum_{i=1}^{n} p_i}{\\sum_{i=1}^{n} p_0}

    :param p0: Base price vector.
    :param p1: Current price vector.
    :param q: Arbitrary quantity vector.
    """
    return np.sum(p1 * q) / np.sum(p0 * q)


def fisher(
    p0: np.array, 
    p1: np.array,
    q0: np.array,
    q1: np.array,
) -> float:
    """
    Fisher bilateral index, using price and quantity information.
    
    .. math::
        \\text{Fisher} = \\frac{\\sum_{i=1}^{n} p_i}{\\sum_{i=1}^{n} p_0}

    :param p0: Base price vector.
    :param p1: Current price vector.
    :param q0: Base quantity vector.
    """
    return np.sqrt(laspeyres(p0, p1, q0) * paasche(p0, p1, q1))

def tornqvist(
    p0: np.array,
    p1: np.array,
    q0: np.array,
    q1: np.array,
) -> float:
    """
    Torqvist bilateral index, using price and quantity information.
    
    .. math::
        \\text{Torqvist} = \\frac{\\sum_{i=1}^{n} p_i}{\\sum_{i=1}^{n} p_0}

    :param p0: Base price vector.
    :param p1: Current price vector.
    :param q0: Base quantity vector.
    :param q1: Current quantity vector.
    """
    s0 = (p0 * q0) / np.sum(p0 * q0)
    s1 = (p1 * q1) / np.sum(p1 * q1)
    return np.prod((p1 / p0) ** (0.5 * (s0 + s1)))

def dutot(
    p0: np.array, 
    p1: np.array
) -> float:
    """
    Dutot bilateral index, using price information.
    
    .. math::
        \\text{Dutot} = \\frac{\\sum_{i=1}^{n} p_i}{\\sum_{i=1}^{n} p_0}

    :param p0: Base price vector.
    :param p1: Current price vector.
    """
    return np.sum(p1)/np.sum(p0)

In [34]:
laspeyres(p0,p1, q0)

1.7

In [37]:
paasche(p0,p1,q1)

1.3

In [42]:
fisher(p0,p1,q0,q1)

1.4866068747318506

In [41]:
tornqvist(p0,p1,p0,q1)

1.4115461908618467